# Formato del dataset de nubes

En este notebook se va a limpiar el dataset de las nubes de forma que nos quedemos solamente con **Cirrus, Altocumulus, Altostratus** y **Stratocumulo**.

Se va a guardar un dataframe de `pandas` en el disco en formato `.pkl` con las clases anteriormente mencionadas.

Cada imagen del dataset (256x256) va a ser partida en patches de (32x32). Estos patches van a ser etiquetados como nubes (dependiendo de su clase) o como cielo.

In [2]:
from sklearn.externals import joblib

import pandas as pd
import numpy as np
import netCDF4

In [3]:
clouds_nc = netCDF4.Dataset('data/TSIdatasetToCarlosIIISet600_20160609.nc')